In [1]:
!pip install mtcnn

In [ ]:
import numpy as np 
import pandas as pd 
import cv2
import os
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt
from keras.models import load_model

In [67]:
facenet=load_model('Jupyter/Photo-Clusterer-Project/Facenet-Model/facenet_keras.h5')
detector_grp=MTCNN(min_face_size = 10,
                   steps_threshold = [0.4,0.5,0.5],
                   scale_factor = 0.5)
detector_single=MTCNN(
    steps_threshold = [0.7,0.8,0.8],
    scale_factor= 0.8)

In [84]:
s=np.load('Jupyter/Photo-Clusterer-Project/Embedds_of_dataset/data.npy',allow_pickle=True)
photos_embedd_dict=s[()]

In [72]:

def faces_extracter(img,grp_or_single='single'):
    
    if grp_or_single == 'grp':
        face_box=detector_grp.detect_faces(img)
    else:
        face_box=detector_single.detect_faces(img)
       
    embedd_faces=[]
    
    for i in range(len(face_box)):
        
        x1,y1,w,h=face_box[i]['box']
        cropped_face=img[y1:y1+h,x1:x1+w]
        cropped_face=cv2.resize(cropped_face,(160,160))
        cropped_face=cropped_face/255
        X=np.expand_dims(cropped_face,axis=0)
        embedd=facenet.predict(X,batch_size=3)
        embedd_faces.append(embedd)
        
    return embedd_faces

In [ ]:
#create empty array
#photos_embedd_dict=dict()

# This step is runned already and "photos_embedd_dict" dictionary is created
# We can directly load the file from cell number 4
'''
for i in os.listdir('Jupyter/Photo-Clusterer-Project/Images'):
    path='/Jupyter/Photo-Clusterer-Project/Images/'+str(i)
    img=cv2.imread(path)
    list_of_faces=faces_extracter(img,'grp')
    photos_embedd_dict['/Jupyter/Photo-Clusterer-Project/Images/'+str(i)]=list_of_faces
    
np.save('data',photos_embedd_dict)
'''

In [85]:
def photo_extracter(selfie):    
    selfie_input='/Jupyter/Photo-Clusterer-Project/Selfie/' + str(selfie)
    selfie_input=cv2.imread(selfie_input)
    selfie_embedd=faces_extracter(selfie_input)
    selfie_embedd=selfie_embedd[0][0]
    

    for i in photos_embedd_dict:
        for j in range(len(photos_embedd_dict[i])):
            embd=photos_embedd_dict[i][j][0]
            result=np.dot(embd,selfie_embedd) / (np.linalg.norm(embd) * np.linalg.norm(selfie_embedd))
            
            if result > 0.68:
                display_image=cv2.imread(i)
                plt.imshow(display_image)
                plt.show()
                break
            

In [ ]:
# Enter your selfie here

selfie=input('Enter your selfie file here')

photo_extracter(selfie)